# ABC for FMC agent-based modelling in demon/warlock

## Procedure
1) Sample a parameter set $\theta$ from the prior
2) Sample a random seed $r$ for the simulation
3) Run the model until the generation limit is reached
4) Repeat from step 2 until the number of simulated glands is the same as in the data
5) Compute wasserstein distances $W(X, X')$ of the simulations to the observed data averaged over all glands
6) If $W(X, X') < \epsilon$, accept $\theta$ as a sample from the posterior distribution
7) Go to 1

In [2]:
import numpy as np
import os
import pandas as pd
import subprocess

from scipy.stats import wasserstein_distance

In [1]:
data_dir = "/Users/vesheljinn/Documents/github_repos/methABC/data/crc_with_healthy_labels/"
sims_dir = "/Users/vesheljinn/Documents/github_repos/methABC/data/sims/"

In [3]:
def run_sim(config_file_path, warlockdir):
    command = ['bash', 'warlock.sh', '-c', config_file_path, '-e', 'local']
    subprocess.run(command, cwd=warlockdir)

Post-processing functions:

In [4]:
def load_data(filename):
    data = pd.read_csv(filename)
    return data

def load_sim(filename):
    sim = pd.read_csv(filename)
    return sim

def distance(sim, data):
    return wasserstein_distance(sim, data)

Iterate over each sim and data set, calculate distance, create matrix of Wasserstein distances:

In [5]:
def get_priors():
    mu = np.random.uniform(0.00001, 0.001)
    s = np.random.uniform(0, 0.4)
    return mu, s

def edit_param_file(oldfilepath, newfilepath):
    with open(oldfilepath, 'r') as file:
        data = yaml.safe_load(file)
    priors = get_priors()
    data['demon_mu_driver_birth'] = priors[0]
    data['demon_s_driver_birth'] = priors[1]

    with open(newfilepath, 'w') as file:
        yaml.dump(data, file)
    return

In [ ]:
simulation_files = os.listdir(sims_dir)

wass_matrix = list()
for simulation_file in simulation_files:
    simulation_path = os.path.join(sims_dir, simulation_file)
    simulation_data = load_data(simulation_path)
    tmp = []
    for data_file in data_files:
        data_path = os.path.join(data_dir, data_file)
        data_set = load_data(data_path)
        
        tmp.append(distance(simulation_data, data_set))
    wass_matrix.append(tmp)